# **Análisis Historias clínicas de pacientes con cancer de pulmón**

Presentado por:

* Mayra Erazo
* Yeraldin Tafur
* Roberto Ceballos
* Katheryn Sanchez

<p align="justify">
El cáncer de pulmón es uno de los más mortales a nivel mundial. La mayoría de los casos se diagnostican en etapas avanzadas, cuando el tratamiento es menos efectivo. Cuando este tipo de cáncer se detecta en etapas iniciales, las tasas de supervivencia aumentan considerablemente. Por eso, la predicción y el tamizaje son claves para salvar vidas. Además, el tratamiento en etapas avanzadas es más costoso y con menos probabilidades de éxito. Predecirlo a tiempo puede reducir los costos para los sistemas de salud. Es por ello que se hace necesario contar con herramientas de analítica que permitan interpretar de una manera sencilla la información que se recolecta en las historias clínicas correspondiente a pacientes relacionados con este diagnóstico, una de esas herramientas es la extracción de información de texto abierto de historias clínicas y su posterior estructuración.</p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Iniciamos este analisis instalando las librerías necesarias para el entrenamiento del modelo.

In [ ]:
!pip install datasets transformers
!pip install seqeval
!pip install -U datasets evaluate
!pip install -U huggingface_hub
!pip install tqdm

Ahora bien procedemos a cargar los datos, los cuales se encuentran en formato CSV. Antes de cargar estos datos creamos una función llamada convertir_csv_bio_from_sentence_column, la cual como su nombre lo indica tiene la función de convertir archivos CSV a formato .bio, el es el formato que acepta este tipo de modeos.

In [ ]:
##Leer archivo .csv y convertirlo en .bio
import pandas as pd

def convertir_csv_bio_from_sentence_column(csv_path, output_path):
    df = pd.read_csv(csv_path)

    # Crear una columna que marque cuándo empieza una nueva oración
    df['new_sentence'] = df['Sentence #'].notna()

    with open(output_path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            if row['new_sentence'] and i != 0:
                f.write('\n')  # Línea en blanco entre oraciones
            f.write(f"{row['Word']} {row['Tag']}\n")


In [ ]:
convertir_csv_bio_from_sentence_column("sentences_train.csv", "training.bio")
convertir_csv_bio_from_sentence_column("sentences_test (3).csv", "testing.bio")
convertir_csv_bio_from_sentence_column("sentences_dev (2).csv", "validation.bio")


In [ ]:
from datasets import DatasetDict, Dataset, Features, Sequence, Value, ClassLabel
from collections import defaultdict
from pathlib import Path

A continuación se crea una función que lee archivos en formato .bio. Esta función primero lee el archivo, y separa las lineas teniendo en cuenta los espacios, para poder almacener en una lista separada los tokens y en otra las etiquetas. Además, se crea una función que permite obtener el diccionario basado en las etiquetas que se encuentran en los arhivos de entrenamiento.

In [ ]:
def leer_archivo_bio(archivo_bio):
    """Lee un archivo .bio y devuelve un diccionario con tokens y etiquetas."""
    datos = defaultdict(list)
    with open(archivo_bio, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    tokens = []
    labels = []
    for num_linea, linea in enumerate(lineas, start=1):
        linea = linea.strip()
        if linea:
            partes = linea.split()
            if len(partes) != 2:
                raise ValueError(f"Error en línea {num_linea}: '{linea}'. Se esperaban 2 elementos.")
            palabra, etiqueta = partes
            tokens.append(palabra)
            labels.append(etiqueta)
        else:
            if tokens and labels:
                datos["tokens"].append(tokens)
                datos["ner_tags"].append(labels)
                tokens = []
                labels = []

    if tokens and labels:
        datos["tokens"].append(tokens)
        datos["ner_tags"].append(labels)

    return datos


def cargar_datasets_bio(rutas_archivos):
    """Carga archivos .bio y devuelve un DatasetDict."""
    datasets = {}
    for nombre, ruta in rutas_archivos.items():
        datos = leer_archivo_bio(ruta)
        datasets[nombre] = Dataset.from_dict(datos)

    return DatasetDict(datasets)



La siguiente función devuelve un conjunto de etiquetas únicas y las ordena.

In [ ]:
# PASO 1: Despues de cargar los datos, primero se detecta todas las etiquetas únicas
def detectar_etiquetas_unicas(rutas_archivos):
    """Detecta automáticamente todas las etiquetas únicas en los archivos."""
    todas_etiquetas = set()

    for ruta in rutas_archivos.values():
        with open(ruta, 'r', encoding='utf-8') as f:
            for linea in f:
                linea = linea.strip()
                if linea:
                    partes = linea.split()
                    if len(partes) == 2:
                        _, etiqueta = partes
                        todas_etiquetas.add(etiqueta)

    # Ordenamos las etiquetas para que 'O' sea la última
    etiquetas_ordenadas = sorted(todas_etiquetas - {'O'}) + ['O']
    return etiquetas_ordenadas

También señalamos la ruta de los archivos transformados que estaremos usando en este ejercicio.

In [ ]:
# Se definen los nombres de las rutas (paths) de los archivos .bio
rutas_archivos = {
    "train": "training.bio",
    "test": "testing.bio",
    "valid": "validation.bio"
}


Utilizamos las funciones anteriormente construidas para obtener las etiquetas, el diccionario y los datasets.

In [ ]:

# Detectar automáticamente todas las etiquetas
LABELS = detectar_etiquetas_unicas(rutas_archivos)
print("Etiquetas detectadas:", LABELS)

# Cargar los datasets
dataset_dict = cargar_datasets_bio(rutas_archivos)

# Definir la estructura de features con las etiquetas detectadas
features = Features({
    "tokens": Sequence(Value("string")),
    "ner_tags": Sequence(ClassLabel(names=LABELS))
})

# Aplicar el casting a cada split
for split in dataset_dict:
    dataset_dict[split] = dataset_dict[split].cast(features)

# Mostrar información del dataset
print("\nDataset cargado correctamente:")
print(dataset_dict)

# Mostrar un ejemplo del conjunto de entrenamiento
print("\nEjemplo del train:")
print(dataset_dict["train"][0])

# Mostrar las características del dataset
print("\nCaracterísticas del dataset:")
print(dataset_dict["train"].features)

Etiquetas detectadas: ['B_CANCER_CONCEPT', 'B_CHEMOTHERAPY', 'B_DATE', 'B_DRUG', 'B_FAMILY', 'B_FREQ', 'B_IMPLICIT_DATE', 'B_INTERVAL', 'B_METRIC', 'B_OCURRENCE_EVENT', 'B_QUANTITY', 'B_RADIOTHERAPY', 'B_SMOKER_STATUS', 'B_STAGE', 'B_SURGERY', 'B_TNM', 'I_CANCER_CONCEPT', 'I_DATE', 'I_DRUG', 'I_FAMILY', 'I_FREQ', 'I_IMPLICIT_DATE', 'I_INTERVAL', 'I_METRIC', 'I_OCURRENCE_EVENT', 'I_SMOKER_STATUS', 'I_STAGE', 'I_SURGERY', 'I_TNM', 'O']


Casting the dataset:   0%|          | 0/9788 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2496 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2758 [00:00<?, ? examples/s]


Dataset cargado correctamente:
DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9788
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2496
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2758
    })
})

Ejemplo del train:
{'tokens': ['Abuela', 'materna', 'con', 'cancer', 'de', 'mama', 'a', 'los', '70', 'años', '.'], 'ner_tags': [4, 19, 29, 0, 16, 16, 29, 29, 10, 8, 29]}

Características del dataset:
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['B_CANCER_CONCEPT', 'B_CHEMOTHERAPY', 'B_DATE', 'B_DRUG', 'B_FAMILY', 'B_FREQ', 'B_IMPLICIT_DATE', 'B_INTERVAL', 'B_METRIC', 'B_OCURRENCE_EVENT', 'B_QUANTITY', 'B_RADIOTHERAPY', 'B_SMOKER_STATUS', 'B_STAGE', 'B_SURGERY', 'B_TNM', 'I_CANCER_CONCEPT', 'I_DATE', 'I_DRUG', 'I_FAMILY', 'I_FREQ', 'I_IMPLICIT_DATE', 'I_INTERVAL', 'I_METRIC', 'I_OCUR

In [ ]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9788
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2496
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2758
    })
})

llamamos el diccionario.

In [ ]:
x = dataset_dict["train"].features[f"{task}_tags"].feature.names
print(x)

['B_CANCER_CONCEPT', 'B_CHEMOTHERAPY', 'B_DATE', 'B_DRUG', 'B_FAMILY', 'B_FREQ', 'B_IMPLICIT_DATE', 'B_INTERVAL', 'B_METRIC', 'B_OCURRENCE_EVENT', 'B_QUANTITY', 'B_RADIOTHERAPY', 'B_SMOKER_STATUS', 'B_STAGE', 'B_SURGERY', 'B_TNM', 'I_CANCER_CONCEPT', 'I_DATE', 'I_DRUG', 'I_FAMILY', 'I_FREQ', 'I_IMPLICIT_DATE', 'I_INTERVAL', 'I_METRIC', 'I_OCURRENCE_EVENT', 'I_SMOKER_STATUS', 'I_STAGE', 'I_SURGERY', 'I_TNM', 'O']


Ahora usamos el modelo BERT para alinear las etiquetas y tokenizar teniendo en cuenta el diccionario y las funciones previas.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True
)

Map:   0%|          | 0/9788 [00:00<?, ? examples/s]

Map:   0%|          | 0/2496 [00:00<?, ? examples/s]

Map:   0%|          | 0/2758 [00:00<?, ? examples/s]

A continuación, definimos la tarea de procesamiento de lenguaje natural de reconocimiento de entidades nombradas "ner", definimos el modelo a utilizar y el tamaño del lote.

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 8

In [ ]:
label_list = dataset_dict["train"].features[f"{task}_tags"].feature.names
label_list

['B_CANCER_CONCEPT',
 'B_CHEMOTHERAPY',
 'B_DATE',
 'B_DRUG',
 'B_FAMILY',
 'B_FREQ',
 'B_IMPLICIT_DATE',
 'B_INTERVAL',
 'B_METRIC',
 'B_OCURRENCE_EVENT',
 'B_QUANTITY',
 'B_RADIOTHERAPY',
 'B_SMOKER_STATUS',
 'B_STAGE',
 'B_SURGERY',
 'B_TNM',
 'I_CANCER_CONCEPT',
 'I_DATE',
 'I_DRUG',
 'I_FAMILY',
 'I_FREQ',
 'I_IMPLICIT_DATE',
 'I_INTERVAL',
 'I_METRIC',
 'I_OCURRENCE_EVENT',
 'I_SMOKER_STATUS',
 'I_STAGE',
 'I_SURGERY',
 'I_TNM',
 'O']

A continuación cargamos el modelo pre entrenado.


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

Ahora lo que hacemos es configurar todos los hiperparámetros y opciones para entrenar un modelo, desde definir donde se guardará el modelo en hugginface, las epocas con las que se va a entrenar, la tasa de aprendizaje, el tamaño de los lotes,el factor para evitar sobre ajustes, entre otros.

In [ ]:
token = getpass("Introduce tu token de Hugging Face: ")
login(token)

model_bert_base = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_bert_base}-finetuned-{task}-pulmon",
    eval_strategy = "epoch", # Changed from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    hub_token=token



)

Se crea una función para evaluar el modelo obteniendo las métricas de rendimiento con las etiquetas predichas.

In [ ]:
try:
    from datasets import load_metric  # Para versiones antiguas
    metric = load_metric("seqeval")
except ImportError:
    from evaluate import load  # Para versiones nuevas
    metric = load("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Validación (eval_dataset): Se usa durante el entrenamiento para:

Ajustar hiperparámetros

Detener el entrenamiento temprano (early stopping)

Monitorizar el progreso

Finalmente entrenamos el modelo.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-68-08bf5cda74a4>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alejandra-erazo (alejandra-erazo-universidad-del-valle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.157200,0.087831,0.920056,0.947696,0.933671,0.977515
2,0.094100,0.079478,0.920626,0.957870,0.938878,0.978111
3,0.065300,0.081195,0.930620,0.963256,0.946656,0.980472
4,0.056300,0.079458,0.934434,0.962059,0.948045,0.980610
5,0.038900,0.087939,0.935720,0.963495,0.949404,0.981343
6,0.035400,0.091601,0.929275,0.959306,0.944052,0.980243
7,0.029300,0.089130,0.938541,0.957750,0.948048,0.981893
8,0.026100,0.094839,0.935015,0.954039,0.944431,0.980518
9,0.021500,0.098173,0.939840,0.955476,0.947593,0.981572
10,0.018400,0.099358,0.938660,0.956074,0.947287,0.981435


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_METRIC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_QUANTITY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DRUG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_FREQ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_FREQ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence

TrainOutput(global_step=12240, training_loss=0.06465053893382253, metrics={'train_runtime': 2478.0967, 'train_samples_per_second': 39.498, 'train_steps_per_second': 4.939, 'total_flos': 3773798538017760.0, 'train_loss': 0.06465053893382253, 'epoch': 10.0})

Buenas prácticas:

No uses test para tomar decisiones: Solo para la evaluación final

Usa validación para ajustes: Early stopping, learning rate, etc.

Guarda test para el final: Como si fuera datos "reales" que el modelo nunca ha visto

In [ ]:
test_metrics = trainer.evaluate(tokenized_datasets["test"])
print("\n" + "="*50)
print(f"Resultados finales en conjunto de test:")
print(f"F1-score: {test_metrics['eval_f1']:.3f}")
print(f"Precisión: {test_metrics['eval_precision']:.3f}")
print(f"Recall: {test_metrics['eval_recall']:.3f}")
print("="*50)


Resultados finales en conjunto de test:
F1-score: 0.933
Precisión: 0.917
Recall: 0.949


De las predicciones realizadas con el modelo el 91.7% fueron realizadas correctamente, también el modelo logra identificar el 95% de las etiquetas y presenta un f1-score de 93% lo que muestra un buen equilibrio entre las entidades encontradas y los verdaderos positivos.
<p align="justify">
Con las métricas anteriores, podemos observar que el modelo tiene un rendimiento muy alto para etiquetar texto clínico en pacientes con cáncer de pulmón. El alto valor de recall sugiere que el modelo es especialmente eficaz para detectar las etiquetas relevantes. Además, mantiene un buen control sobre los errores en las predicciones, lo cual se refleja en su elevada precisión.
<p align="justify">
En resumen, el modelo muestra un equilibrio adecuado, manteniendo bajos niveles de falsos negativos y falsos positivos. Esto es particularmente importante, ya que una alta tasa de verdaderos positivos y verdaderos negativos permite focalizar el análisis de los datos y generar conclusiones más precisas sobre aspectos relacionados con el cáncer de pulmón.

In [ ]:
trainer.push_to_hub()

Uploading...:   0%|          | 0.00/436M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mayra13/bert-base-uncased-finetuned-ner-pulmon/commit/95197b1911bcaa1b5395aca059c9112d94ad3bdf', commit_message='End of training', commit_description='', oid='95197b1911bcaa1b5395aca059c9112d94ad3bdf', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mayra13/bert-base-uncased-finetuned-ner-pulmon', endpoint='https://huggingface.co', repo_type='model', repo_id='Mayra13/bert-base-uncased-finetuned-ner-pulmon'), pr_revision=None, pr_num=None)

In [ ]:
label_names =  dataset_dict["train"].features["ner_tags"].feature.names
label_names

['B_CANCER_CONCEPT',
 'B_CHEMOTHERAPY',
 'B_DATE',
 'B_DRUG',
 'B_FAMILY',
 'B_FREQ',
 'B_IMPLICIT_DATE',
 'B_INTERVAL',
 'B_METRIC',
 'B_OCURRENCE_EVENT',
 'B_QUANTITY',
 'B_RADIOTHERAPY',
 'B_SMOKER_STATUS',
 'B_STAGE',
 'B_SURGERY',
 'B_TNM',
 'I_CANCER_CONCEPT',
 'I_DATE',
 'I_DRUG',
 'I_FAMILY',
 'I_FREQ',
 'I_IMPLICIT_DATE',
 'I_INTERVAL',
 'I_METRIC',
 'I_OCURRENCE_EVENT',
 'I_SMOKER_STATUS',
 'I_STAGE',
 'I_SURGERY',
 'I_TNM',
 'O']

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'_CANCER_CONCEPT': {'precision': np.float64(0.8815426997245179),
  'recall': np.float64(0.9288824383164006),
  'f1': np.float64(0.9045936395759717),
  'number': np.int64(689)},
 '_CHEMOTHERAPY': {'precision': np.float64(0.9794871794871794),
  'recall': np.float64(1.0),
  'f1': np.float64(0.9896373056994818),
  'number': np.int64(191)},
 '_DATE': {'precision': np.float64(0.9808184143222506),
  'recall': np.float64(0.9845956354300385),
  'f1': np.float64(0.982703395259449),
  'number': np.int64(779)},
 '_DRUG': {'precision': np.float64(0.9178272980501393),
  'recall': np.float64(0.9762962962962963),
  'f1': np.float64(0.946159368269921),
  'number': np.int64(675)},
 '_FAMILY': {'precision': np.float64(0.9865771812080537),
  'recall': np.float64(1.0),
  'f1': np.float64(0.9932432432432432),
  'number': np.int64(147)},
 '_FREQ': {'precision': np.float64(0.8932584269662921),
  'recall': np.float64(0.9875776397515528),
  'f1': np.float64(0.9380530973451326),
  'number': np.int64(161)},
 '_I

<p align="justify">
Ahora evaluamos la efectividad del modelo para predecir cada una de las etiquetas. Observamos que:


* <p align="justify"> Rendimiento alto (F1 > 95%):El modelo es muy sólido en las etiquetas más clínicas y estructuradas, mostrando un alto equilibrio y una elevada tasa de verdaderos positivos y verdaderos negativos. A continuación, se listan las etiquetas cuyo F1-score es superior al 95%: FAMILY, CHEMOTHERAPY, DATE, STAGE y QUANTITY. Cabe resaltar que la etiqueta mejor predicha fue CHEMOTHERAPY. Para esta categoría, no se presentaron falsos positivos ni falsos negativos, ya que alcanzó un recall de 1.0. Sin embargo, es importante considerar que la información puede estar desbalanceada, y podrían existir menos casos asociados a esta etiqueta, lo cual podría influir en el rendimiento observado.


* <p align="justify"> Rendimiento medio-alto (F1 entre 90% y 94%): METRIC, DRUG, TNM, FREQ,RADIOTHERAPY, CANCER_CONCEPT, el modelo presentó un redimiento aceptable pero con posibilidad de mejora y afinamiento, ya que el F1 score oscilo al rededor de 90% a 94%, observemos que en estas etiquetas se encuentran algunas de suma importancia como el conceto de cancer, métricas y drogas suministradas, si bien, las métricas presentadas para estas etiquetas no son muy bajas, hay una pequeña posibilidad de mejora esto con el fin de precisar en estos puntos de análisis que son de suma importancia.


* <p align="justify"> Rendimiento bajo (F1 < 90%): Entre las etiquetas más débiles con F1-score inferior al 90% encontramos: STATUS_SMOKER,INTERVAL, SUGERY, OCURRENCE_EVENT, IMPLICIT_DATE, en estas etiquetas se encuentra una oportunidad de mejora importante, en especial para la etiqueta IMPLICIT_DATE la cual presenta el F1-Score más bajo del 53%, lo que indica que estas fechas son dificles de detectar, o no hay un gran número de etiquetas para lograr un correcto entremiento.



<p align="justify">
A manera de conclusión podemos decir que, contar con modelos de lenguaje natural capaces de etiquetar automáticamente textos clínicos, como historias médicas de pacientes con cáncer de pulmón, representa un avance significativo en la gestión y análisis de datos no estructurados.

<p align="justify">
Estos textos suelen contener información valiosa pero dispersa, escrita en lenguaje natural por distintos profesionales de la salud, con variaciones en estilo y terminología. Automatizar la extracción de entidades específicas permite transformar esa información en datos estructurados, accesibles y útiles para la toma de decisiones clínicas, administrativas e investigativas.

<p align="justify">
Con el modelo implementado, es posible:

<p align="justify">
-Detectar antecedentes familiares, lo cual es clave para evaluar el riesgo genético.
<p align="justify">
-Extraer información sobre quimioterapia, lo que permite identificar tratamientos más efectivos.
<p align="justify">
-Capturar fechas explícitas, fundamentales para construir líneas de tiempo y estudiar la evolución de los pacientes.
<p align="justify">
-Identificar estadios frecuentes del cáncer, para apoyar el análisis clínico.
<p align="justify">
-Reconocer medicamentos y dosis, permitiendo asociarlos a reacciones adversas o eficacia terapéutica.
<p align="justify">
-Extraer tamaños tumorales y métricas clínicas relevantes.
<p align="justify">
-Reconocer conceptos oncológicos clave, para mejorar la codificación y estandarización de registros médicos.
<p align="justify">
Todo esto permite aprovechar al máximo la información contenida en las historias clínicas, transformándolas en una fuente rica para el análisis, la investigación y la mejora del cuidado del paciente.</p>

